In [2]:
import pandas as pd
import numpy as np
import selenium
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error

# 데이터 읽어오기
## DataFrame
### - data : 본 데이터

In [ ]:
#데이터 읽어오기
data = pd.read_csv('/Users/parkjeongwon/Downloads/jeju_data_ver1/201901-202003.csv')

In [ ]:
# 필요없는 칼럼 제거
data = data.drop(['CARD_CCG_NM', 'HOM_SIDO_NM', 'HOM_CCG_NM', 'FLC', 'CNT'], axis = 1)

# 아이디어
1. 월/나이/성별에 따라 업종에 쓴 돈의 비율, 지역별로 쓴 돈의 비율이 전년도 해당 월과 동일하거나 비슷할 것이라 예측(개인적 경험 및 주변 지인들 조사)
 - 업종에 쓴 돈의 비율 = 월/나이/성별/업종에 따른 AMT / 월,나이,성별에 따른 AMT
 - 지역에 쓴 돈의 비율 = 월/나이/성별/지역에 따른 AMT / 월,나이,성별에 따른 AMT
2. 전 달의 카드사용량과 관계가 있을 것이다.
3. 월/지역/나이/성별에 따른 인구수와 관련이 있을 것이다(해당 지역에 사는 사람이 많을수록 해당 지역 소비가 클 것이라 생각)
4. 월/지역별 축제 수와 관계가 있을 것이다.
5. 해당 월의 공휴일 수와 관계 있을 것이다.
6. 월, 지역, 업종, 나이, 성별에 따른 AMT를 예측하고 그에 따라 월,지역,업종별 AMT 계산

# 1. 월
##  - month / 월

In [3]:
# 년월을 월로 바꿔 줄 함수 정의(ex, 201901 -> 1)
def month(x):
    a = str(x)[4:]
    return int(a)

In [ ]:
data['month'] = data['REG_YYMM'].apply(lambda x : month(x))

# 2. 월/나이/성별에 따른 업종 별 카드사용량 비율, 월/나이/성별에 따른 지역 별 카드사용량 비율
## str
### - standard : 월,나이,성별
### - standard_STD_CLSS_NM : 월,나이,성별, 업종
### - standard_CARD_SIDO_NM : 월,나이,성별,지역
## int
### - standard_AMT : 월/나이/성별에 따른 카드사용량 
### - standard_STD_CLSS_NM_AMT : 월/나이/성별/업종에 따른 카드사용량
### - standard_CARD_SIDO_AMT : 월/나이/성별/지역에 따른 카드사용량
### - standard_STD_CLSS_ratio : 월/나이/성별에 따른 업종 별 카드사용량 비율
### - standard_CARD_SIDO_ratio : 월/나이/성별에 따른 지역 별 카드사용량 비율


In [ ]:
# 월 나이 성별 / 업종에 쓴 돈의 비율, 지역에 쓴 돈의 비율의 분모를 구하기 위해
data['month'] = data['month'].astype(str)
data['AGE'] = data['AGE'].astype(str)
data['SEX_CTGO_CD'] = data['SEX_CTGO_CD'].astype(str)

data['standard'] = data['month'] + ',' + data['AGE'] + ',' + data['SEX_CTGO_CD']

# 월 나이 성별 업종 / 업종에 쓴 돈의 비율의 분자를 구하기 위해
data['month'] = data['month'].astype(str)
data['AGE'] = data['AGE'].astype(str)
data['SEX_CTGO_CD'] = data['SEX_CTGO_CD'].astype(str)
data['STD_CLSS_NM'] = data['STD_CLSS_NM'].astype(str)

data['standard_STD_CLSS_NM'] = data['month'] + ',' + data['AGE'] + ',' + data['SEX_CTGO_CD'] + ',' + data['STD_CLSS_NM']

# 월 나이 성별 지역 / 지역에 쓴 돈의 비율의 분자를 구하기 위해
data['month'] = data['month'].astype(str)
data['AGE'] = data['AGE'].astype(str)
data['SEX_CTGO_CD'] = data['SEX_CTGO_CD'].astype(str)
data['CARD_SIDO_NM'] = data['CARD_SIDO_NM'].astype(str)

data['standard_CARD_SIDO_NM'] = data['month'] + ',' + data['AGE'] + ',' + data['SEX_CTGO_CD'] + ',' + data['CARD_SIDO_NM']


In [ ]:
# 월/나이/성별에 따른 AMT
a = data.groupby(['standard'])['AMT'].agg('sum').reset_index()
a.columns = ['standard', 'standard_AMT']

data = pd.merge(data, a, how='left', on='standard')

# 월/나이/성별에 따른 업종별 AMT
a = data.groupby(['standard_STD_CLSS_NM'])['AMT'].agg('sum').reset_index()
a.columns = ['standard_STD_CLSS_NM', 'standard_STD_CLSS_NM_AMT']

data = pd.merge(data, a, how='left', on='standard_STD_CLSS_NM')

# 월/나이/성별에 따른 지역별 AMT
a = data.groupby(['standard_CARD_SIDO_NM'])['AMT'].agg('sum').reset_index()
a.columns = ['standard_CARD_SIDO_NM', 'standard_CARD_SIDO_AMT']

data = pd.merge(data, a, how='left', on='standard_CARD_SIDO_NM')

In [ ]:
# 계산을 위해 정수로 바꿔주기
data['standard_AMT'] = data['standard_AMT'].astype(int)
data['standard_STD_CLSS_NM_AMT'] = data['standard_STD_CLSS_NM_AMT'].astype(int)
data['standard_CARD_SIDO_AMT'] = data['standard_CARD_SIDO_AMT'].astype(int)

#월 / 나이 / 성별에 따른 업종별 비율
data['standard_STD_CLSS_ratio'] = data['standard_STD_CLSS_NM_AMT'] / data['standard_AMT']

#월 / 나이 / 성별에 따른 지역별 비율
data['standard_CARD_SIDO_ratio'] = data['standard_CARD_SIDO_AMT'] / data['standard_AMT']

# 3. 전 달 카드사용량
## columns
### - for_last : 년월,업종,지역 
### - last_month_AMT : 년월,업종,지역에 따른 카드사용량
### - last_month : for_last 한 달 뒤 년월, 업종,지역(ex, for_last = 201901,강원,소비업 -> last_month = 201902,강원,소비업)
### - base_a : 업종,지역
### - last_month_AMT : 전 달 카드 사용량

## DataFrame
### - data_temp : [last_month(년월/업종/지역)(201902~202004), last_month_AMT(전 달 카드사용량)]
### - data_201901 : [last_month(201901/업종/지역), last_month_AMT(전 달 카드사용량)]
### - data_last : [last_month(년월/업종/지역)(201901~202004), last_month_AMT_r(전 달 카드사용량)]
### - check : data['for_last'] (본 데이터의 년월,업종,지역)
### - check_ok : data['for_last']와 data_last['last_month]에 둘 다 들어 있는 값들
### - check_no : data['for_last']에는 있지만 data_last['last_month]에 없는 값들
### - data : 본 데이터 + 'last_month_AMT_r' (전 달 카드사용량 추가)

In [ ]:
#년월, 업종, 카드사용장소
data['REG_YYMM'] = data['REG_YYMM'].astype(str)
data['STD_CLSS_NM'] = data['STD_CLSS_NM'].astype(str)
data['CARD_SIDO_NM'] = data['CARD_SIDO_NM'].astype(str)
data['for_last'] = data['REG_YYMM'] + ',' + data['STD_CLSS_NM'] + ',' + data['CARD_SIDO_NM']

In [ ]:
#년월, 업종, 지역에 따른 AMT값
a = data.groupby(['for_last'])['AMT'].agg('sum').reset_index()
a.columns = ['for_last', 'last_month_AMT']

data = pd.merge(data, a, how='left', on='for_last')

In [ ]:
# for_last의 년월 중 월을 다음달로 변경해주는 함수 정의(ex, 201901,강원,소비업 -> 201902,강원,소비업)
def trans1(x):
    if int(str(x)[:6]) == 201912:
        x = '202001' + str(x)[6:]
    else:
        x = str(int(str(x)[:6])+1) + str(x)[6:]
    return x

In [ ]:
# 이렇게 되면 last_month와 last_month_AMT의 관계는 '년월,업종,지역'과 년월,업종,지역에 따른 전달 카드사용량이 된다.
data['last_month'] = data['for_last'].apply(lambda x : trans1(x))

In [ ]:
# data_temp는 이번달/업종/지역 , 저번달 카드사용량이 된다.
data_temp = data[['last_month', 'last_month_AMT']]
# 중복 행 제거(데이터 처리 속도 증가)
data_temp = data_temp.drop_duplicates()

In [ ]:
# 2019년01월의 경우 전 달인 2018년 12월의 데이터가 필요하나 없으니 2019년01월로 대체
data_201901 = data[data['REG_YYMM'] == '201901']
data_201901 = data_201901[['for_last', 'last_month_AMT']]
data_201901.columns = ['last_month', 'last_month_AMT']

In [ ]:
# data_temp와 data_201901 병합
data_last = pd.concat([data_temp, data_201901], axis = 0)

In [ ]:
#중복되는 행을 제거함으로써 성능을 높인다
data_last = data_last.drop_duplicates()

본 데이터인 data['for_last']에는 있지만 data_last에서는 없는 값을 찾아야 한다. 
data_las는 trans1 함수를 통해 임의적으로 만들어낸 데이터이므로 본 데이터의 값이 없을 수도 있기 때문에 이 과정을 실해

In [ ]:
# data['for_last']에는 값이 있지만 data_last['last_month']에는 없는 데이터 확인
check = data['for_last']
check = check.drop_duplicates()
check_ok = pd.DataFrame(columns = ['last_month'])

for i in range(0, data_last.shape[0]):
    a = data_last['last_month'].iloc[i]
    check_ok = pd.concat([check_ok, check[check.isin([a])]], axis = 0)

check_no = check.drop(check_ok.index, axis = 0)
check_no = pd.DataFrame(check_no)

check_no의 값은 전 달의 카드사용량 데이터가 없기 때문에 지역,업종 별 평균으로 대체하기 위해 'for_last'에서 지역, 업종만을 뽑아내기 위해서 아래의 과정을 실행

In [ ]:
def trans2(x):
    return str(x)[7:]

In [ ]:
check_no['Sectors'] = check_no['for_last'].apply(lambda x : trans2(x))

In [ ]:
# 지역, 업종별 평균을 구하기 위해
data['STD_CLSS_NM'] = data['STD_CLSS_NM'].astype(str)
data['CARD_SIDO_NM'] = data['CARD_SIDO_NM'].astype(str)

data['base_a'] = data['STD_CLSS_NM'] + ',' + data['CARD_SIDO_NM']

# 없는 값은 지역, 업종별 평균으로 대체
data_a = data[['base_a', 'last_month_AMT']]
data_a = data_a.drop_duplicates()
a = data_a.groupby(['base_a'])['last_month_AMT'].agg('mean').reset_index()
a.columns = ['Sectors', 'last_month_AMT']

# 값들에 업종,지역별 평균 값을 넣어줌
check_no = pd.merge(check_no, a, how = 'left', on = 'Sectors')

In [ ]:
# 필요없는 column 제거
check_no = check_no.drop(['Sectors'], axis = 1)
check_no.columns = ['last_month', 'last_month_AMT']

In [ ]:
# 병합
data_last = pd.concat([data_last, check_no], axis = 0)
# 이제 완전한 값들로 이루어진 data_last
data_last.columns = ['for_last', 'last_month_AMT_r']
data_last = data_last.drop_duplicates()

In [ ]:
# 원래의 데이터와 병합
data = pd.merge(data, data_last, how = 'left', on = 'for_last')

In [ ]:
# 결측값 확인
data.isnull().sum()

In [ ]:
# 중간 저장
data.to_csv('data_30%', index = False)

# 4. 월/지역/나이/성별에 따른 인구 수
- 링크 : http://27.101.213.4/ageStatMonth.do
## columns
### - for_last : 년월,업종,지역 

## DataFrame
### - num_data_01_06 : 2019.01~06월/지역/나이/성별에 따른 인구 수
### - num_data_07_12 : 2019.07~12월/지역/나이/성별에 따른 인구 수
### - num_data_20_01_03 : 2020.01~03월/지역/나이/성별에 따른 인구 수
### - num_target : 2020.04월/지역/나이/성별에 따른 인구 수
### - data_age_m : 2019년01월~2020년03월 남성들의 지역/나이별 인구 수(index = 지역, column = 년월,나이)
### - data_m_20_04 : 2020년4월 남성들의 지역/나이별 인구 수(index = 지역, column = 년월,나이)
### - data_age_f : 2019년01월~2020년03월 여성들의 지역/나이별 인구 수(index = 지역, column = 년월,나이)
### - data_f_20_04 : 2020년4월 여성들의 지역/나이별 인구 수(index = 지역, column = 년월,나이)
### - data_po : 2019년01월~2020년03월 지역/나이/성별에 따른 인구 수
### - data_population_target : 2020년04월 지역/나이/성별에 따른 인구 수
### - population : 2019년01월~2020년03월[for_population('년월/지역/나이/성별'), population(조건에 맞는 값)]
### - population_target : 2020년04월[for_population('년월/지역/나이/성별'), population(조건에 맞는 값)]

In [ ]:
num_data_01_06 = pd.read_csv('/Users/parkjeongwon/Downloads/201901_201906_연령별인구현황_월간.csv', encoding='CP949')
num_data_07_12 = pd.read_csv('/Users/parkjeongwon/Downloads/201907_201912_연령별인구현황_월간.csv', encoding='CP949')
num_data_20_01_03 = pd.read_csv('/Users/parkjeongwon/Downloads/202001_202003_연령별인구현황_월간.csv', encoding='CP949')
num_target = pd.read_csv('/Users/parkjeongwon/Downloads/202004_202004_연령별인구현황_월간 (1).csv', encoding='CP949')

In [ ]:
#필요없는 값들 제거
num_data_01_06 = num_data_01_06.drop([0],axis = 0)
num_data_07_12 = num_data_07_12.drop([0],axis = 0)
num_data_20_01_03 = num_data_20_01_03.drop([0],axis = 0)
num_target = num_target.drop([0], axis = 0)

In [ ]:
# 위의 csv 파일들의 값에 ','가 찍혀 있어서 제거하기 위해(ex, 19,802,345 - > 19802345)
def change_(x):
    if len(x) < 5:
        return int(x)
    elif len(x) < 9 and len(x) >= 5:
        return int(str(x)[:-4] + str(x)[-3:])
    elif len(x) >= 9:
        return int(str(x)[:-8] + str(x)[-7:-4] + str(x)[-3:])

데이터들의 행은 지역(서울,강원,.....) 열은 '년월,성별,나이'(2019년01월_남_10~19세,......)으로 구성되고 값들에는 ','가 찍혀있으므로 아래의 방법을 통해 데이터들을 재구성

### 2019년01월~2020년03월 남성

In [ ]:
#19.01
result_age_01 = []
for j in range(0, num_data_01_06.shape[0]):
    temp = []
    for i in ['2019년01월_남_10~19세', '2019년01월_남_20~29세', '2019년01월_남_30~39세',
              '2019년01월_남_40~49세', '2019년01월_남_50~59세', '2019년01월_남_60~69세', '2019년01월_남_70~79세']:
        temp.append(change_(num_data_01_06[i].iloc[j]))
    result_age_01.append(temp)
    
data_age_01 = pd.DataFrame(result_age_01, columns = ['1_10s','1_20s', '1_30s', '1_40s', '1_50s', '1_60s', '1_70s'])

#19.02
result_age_02 = []
for j in range(0, num_data_01_06.shape[0]):
    temp = []
    for i in ['2019년02월_남_10~19세', '2019년02월_남_20~29세', '2019년02월_남_30~39세',
              '2019년02월_남_40~49세', '2019년02월_남_50~59세', '2019년02월_남_60~69세', '2019년02월_남_70~79세']:
        temp.append(change_(num_data_01_06[i].iloc[j]))
    result_age_02.append(temp)
    
data_age_02 = pd.DataFrame(result_age_02, columns = ['2_10s','2_20s', '2_30s', '2_40s', '2_50s', '2_60s', '2_70s'])

#19.03
result_age_03 = []
for j in range(0, num_data_01_06.shape[0]):
    temp = []
    for i in ['2019년03월_남_10~19세', '2019년03월_남_20~29세', '2019년03월_남_30~39세',
              '2019년03월_남_40~49세', '2019년03월_남_50~59세', '2019년03월_남_60~69세', '2019년03월_남_70~79세']:
        temp.append(change_(num_data_01_06[i].iloc[j]))
    result_age_03.append(temp)

data_age_03 = pd.DataFrame(result_age_03, columns = ['3_10s','3_20s', '3_30s', '3_40s', '3_50s', '3_60s', '3_70s'])

#19.04
result_age_04 = []
for j in range(0, num_data_01_06.shape[0]):
    temp = []
    for i in ['2019년04월_남_10~19세', '2019년04월_남_20~29세', '2019년04월_남_30~39세',
              '2019년04월_남_40~49세', '2019년04월_남_50~59세', '2019년04월_남_60~69세', '2019년04월_남_70~79세']:
        temp.append(change_(num_data_01_06[i].iloc[j]))
    result_age_04.append(temp)

data_age_04 = pd.DataFrame(result_age_04, columns = ['4_10s','4_20s', '4_30s', '4_40s', '4_50s', '4_60s', '4_70s'])

#19.05
result_age_05 = []
for j in range(0, num_data_01_06.shape[0]):
    temp = []
    for i in ['2019년05월_남_10~19세', '2019년05월_남_20~29세', '2019년05월_남_30~39세',
              '2019년05월_남_40~49세', '2019년05월_남_50~59세', '2019년05월_남_60~69세', '2019년05월_남_70~79세']:
        temp.append(change_(num_data_01_06[i].iloc[j]))
    result_age_05.append(temp)
    
data_age_05 = pd.DataFrame(result_age_05, columns = ['5_10s','5_20s', '5_30s', '5_40s', '5_50s', '5_60s', '5_70s'])

#19.06
result_age_06 = []
for j in range(0, num_data_01_06.shape[0]):
    temp = []
    for i in ['2019년06월_남_10~19세', '2019년06월_남_20~29세', '2019년06월_남_30~39세',
              '2019년06월_남_40~49세', '2019년06월_남_50~59세', '2019년06월_남_60~69세', '2019년06월_남_70~79세']:
        temp.append(change_(num_data_01_06[i].iloc[j]))
    result_age_06.append(temp)
    
data_age_06 = pd.DataFrame(result_age_06, columns = ['6_10s','6_20s', '6_30s', '6_40s', '6_50s', '6_60s', '6_70s'])

#19.07
result_age_07 = []
for j in range(0, num_data_07_12.shape[0]):
    temp = []
    for i in ['2019년07월_남_10~19세', '2019년07월_남_20~29세', '2019년07월_남_30~39세',
              '2019년07월_남_40~49세', '2019년07월_남_50~59세', '2019년07월_남_60~69세', '2019년07월_남_70~79세']:
        temp.append(change_(num_data_07_12[i].iloc[j]))
    result_age_07.append(temp)
    
data_age_07 = pd.DataFrame(result_age_07, columns = ['7_10s','7_20s', '7_30s', '7_40s', '7_50s', '7_60s', '7_70s'])

#19.08
result_age_08 = []
for j in range(0, num_data_07_12.shape[0]):
    temp = []
    for i in ['2019년08월_남_10~19세', '2019년08월_남_20~29세', '2019년08월_남_30~39세',
              '2019년08월_남_40~49세', '2019년08월_남_50~59세', '2019년08월_남_60~69세', '2019년08월_남_70~79세']:
        temp.append(change_(num_data_07_12[i].iloc[j]))
    result_age_08.append(temp)

data_age_08 = pd.DataFrame(result_age_08, columns = ['8_10s','8_20s', '8_30s', '8_40s', '8_50s', '8_60s', '8_70s'])

#19.09
result_age_09 = []
for j in range(0, num_data_07_12.shape[0]):
    temp = []
    for i in ['2019년09월_남_10~19세', '2019년09월_남_20~29세', '2019년09월_남_30~39세',
              '2019년09월_남_40~49세', '2019년09월_남_50~59세', '2019년09월_남_60~69세', '2019년09월_남_70~79세']:
        temp.append(change_(num_data_07_12[i].iloc[j]))
    result_age_09.append(temp)
    
data_age_09 = pd.DataFrame(result_age_09, columns = ['9_10s','9_20s', '9_30s', '9_40s', '9_50s', '9_60s', '9_70s'])

#19.10
result_age_10 = []
for j in range(0, num_data_07_12.shape[0]):
    temp = []
    for i in ['2019년10월_남_10~19세', '2019년10월_남_20~29세', '2019년10월_남_30~39세',
              '2019년10월_남_40~49세', '2019년10월_남_50~59세', '2019년10월_남_60~69세', '2019년10월_남_70~79세']:
        temp.append(change_(num_data_07_12[i].iloc[j]))
    result_age_10.append(temp)
    
data_age_10 = pd.DataFrame(result_age_10, columns = ['10_10s','10_20s', '10_30s', '10_40s', '10_50s', '10_60s', '10_70s'])

#19.11
result_age_11 = []
for j in range(0, num_data_07_12.shape[0]):
    temp = []
    for i in ['2019년11월_남_10~19세', '2019년11월_남_20~29세', '2019년11월_남_30~39세',
              '2019년11월_남_40~49세', '2019년11월_남_50~59세', '2019년11월_남_60~69세', '2019년11월_남_70~79세']:
        temp.append(change_(num_data_07_12[i].iloc[j]))
    result_age_11.append(temp)
    
data_age_11 = pd.DataFrame(result_age_11, columns = ['11_10s','11_20s', '11_30s', '11_40s', '11_50s', '11_60s', '11_70s'])

#19.12
result_age_12 = []
for j in range(0, num_data_07_12.shape[0]):
    temp = []
    for i in ['2019년12월_남_10~19세', '2019년12월_남_20~29세', '2019년12월_남_30~39세',
              '2019년12월_남_40~49세', '2019년12월_남_50~59세', '2019년12월_남_60~69세', '2019년12월_남_70~79세']:
        temp.append(change_(num_data_07_12[i].iloc[j]))
    result_age_12.append(temp)

data_age_12 = pd.DataFrame(result_age_12, columns = ['12_10s','12_20s', '12_30s', '12_40s', '12_50s', '12_60s', '12_70s'])

#20.01
result_age_20_01 = []
for j in range(0, num_data_20_01_03.shape[0]):
    temp = []
    for i in ['2020년01월_남_10~19세', '2020년01월_남_20~29세', '2020년01월_남_30~39세',
              '2020년01월_남_40~49세', '2020년01월_남_50~59세', '2020년01월_남_60~69세', '2020년01월_남_70~79세']:
        temp.append(change_(num_data_20_01_03[i].iloc[j]))
    result_age_20_01.append(temp)
    
data_age_20_01 = pd.DataFrame(result_age_20_01, columns = ['20_01_10s','20_01_20s', '20_01_30s', '20_01_40s', '20_01_50s', '20_01_60s', '20_01_70s'])

#20.02
result_age_20_02 = []
for j in range(0, num_data_20_01_03.shape[0]):
    temp = []
    for i in ['2020년02월_남_10~19세', '2020년02월_남_20~29세', '2020년02월_남_30~39세',
              '2020년02월_남_40~49세', '2020년02월_남_50~59세', '2020년02월_남_60~69세', '2020년02월_남_70~79세']:
        temp.append(change_(num_data_20_01_03[i].iloc[j]))
    result_age_20_02.append(temp)
    
data_age_20_02 = pd.DataFrame(result_age_20_02, columns = ['20_02_10s','20_02_20s', '20_02_30s', '20_02_40s', '20_02_50s', '20_02_60s', '20_02_70s'])

#20.03
result_age_20_03 = []
for j in range(0, num_data_20_01_03.shape[0]):
    temp = []
    for i in ['2020년03월_남_10~19세', '2020년03월_남_20~29세', '2020년03월_남_30~39세',
              '2020년03월_남_40~49세', '2020년03월_남_50~59세', '2020년03월_남_60~69세', '2020년03월_남_70~79세']:
        temp.append(change_(num_data_20_01_03[i].iloc[j]))
    result_age_20_03.append(temp)

data_age_20_03 = pd.DataFrame(result_age_20_03, columns = ['20_03_10s','20_03_20s', '20_03_30s', '20_03_40s', '20_03_50s', '20_03_60s', '20_03_70s'])

data_age_m = pd.concat([data_age_01, data_age_02, data_age_03, data_age_04, data_age_05, data_age_06, data_age_07, 
           data_age_08, data_age_09, data_age_10, data_age_11, data_age_12, data_age_20_01, data_age_20_02, 
                     data_age_20_03], axis = 1)
data_age_m.index = ['서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종', '경기', '강원', '충북',
                     '충남', '전북', '전남', '경북', '경남', '제주']

### 2020년04월 남성

In [ ]:
#20.04
result_age_m_target = []
for j in range(0, num_target.shape[0]):
    temp = []
    for i in ['2020년04월_남_10~19세', '2020년04월_남_20~29세', '2020년04월_남_30~39세',
            '2020년04월_남_40~49세', '2020년04월_남_50~59세', '2020년04월_남_60~69세',
            '2020년04월_남_70~79세']:
        temp.append(change_(num_target[i].iloc[j]))
    result_age_m_target.append(temp)
data_m_20_04 = pd.DataFrame(result_age_m_target, columns = ['10s', '20s', '30s', '40s', '50s', '60s', '70s'])
data_m_20_04.index = ['서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종', '경기', '강원', '충북',
                     '충남', '전북', '전남', '경북', '경남', '제주']

### 2019년01월~2020년03월 여성

In [ ]:
#19.01
result_age_f_01 = []
for j in range(0, num_data_01_06.shape[0]):
    temp = []
    for i in ['2019년01월_여_10~19세', '2019년01월_여_20~29세', '2019년01월_여_30~39세',
              '2019년01월_여_40~49세', '2019년01월_여_50~59세', '2019년01월_여_60~69세', '2019년01월_여_70~79세']:
        temp.append(change_(num_data_01_06[i].iloc[j]))
    result_age_f_01.append(temp)

data_age_f_01 = pd.DataFrame(result_age_f_01, columns = ['f_01_10s','f_01_20s', 'f_01_30s', 'f_01_40s', 'f_01_50s', 'f_01_60s', 'f_01_70s'])

#19.02
result_age_f_02 = []
for j in range(0, num_data_01_06.shape[0]):
    temp = []
    for i in ['2019년02월_여_10~19세', '2019년02월_여_20~29세', '2019년02월_여_30~39세',
              '2019년02월_여_40~49세', '2019년02월_여_50~59세', '2019년02월_여_60~69세', '2019년02월_여_70~79세']:
        temp.append(change_(num_data_01_06[i].iloc[j]))
    result_age_f_02.append(temp)
    
data_age_f_02 = pd.DataFrame(result_age_f_02, columns = ['f_02_10s','f_02_20s', 'f_02_30s', 'f_02_40s', 'f_02_50s', 'f_02_60s', 'f_02_70s'])

#19.03
result_age_f_03 = []
for j in range(0, num_data_01_06.shape[0]):
    temp = []
    for i in ['2019년03월_여_10~19세', '2019년03월_여_20~29세', '2019년03월_여_30~39세',
              '2019년03월_여_40~49세', '2019년03월_여_50~59세', '2019년03월_여_60~69세', '2019년03월_여_70~79세']:
        temp.append(change_(num_data_01_06[i].iloc[j]))
    result_age_f_03.append(temp)

data_age_f_03 = pd.DataFrame(result_age_f_03, columns = ['f_03_10s','f_03_20s', 'f_03_30s', 'f_03_40s', 'f_03_50s', 'f_03_60s', 'f_03_70s'])
    
#19.04
result_age_f_04 = []
for j in range(0, num_data_01_06.shape[0]):
    temp = []
    for i in ['2019년04월_여_10~19세', '2019년04월_여_20~29세', '2019년04월_여_30~39세',
              '2019년04월_여_40~49세', '2019년04월_여_50~59세', '2019년04월_여_60~69세', '2019년04월_여_70~79세']:
        temp.append(change_(num_data_01_06[i].iloc[j]))
    result_age_f_04.append(temp)

data_age_f_04 = pd.DataFrame(result_age_f_04, columns = ['f_04_10s','f_04_20s', 'f_04_30s', 'f_04_40s', 'f_04_50s', 'f_04_60s', 'f_04_70s'])

#19.05
result_age_f_05 = []
for j in range(0, num_data_01_06.shape[0]):
    temp = []
    for i in ['2019년05월_여_10~19세', '2019년05월_여_20~29세', '2019년05월_여_30~39세',
              '2019년05월_여_40~49세', '2019년05월_여_50~59세', '2019년05월_여_60~69세', '2019년05월_여_70~79세']:
        temp.append(change_(num_data_01_06[i].iloc[j]))
    result_age_f_05.append(temp)
    
data_age_f_05 = pd.DataFrame(result_age_f_05, columns = ['f_05_10s','f_05_20s', 'f_05_30s', 'f_05_40s', 'f_05_50s', 'f_05_60s', 'f_05_70s'])

#19.06
result_age_f_06 = []
for j in range(0, num_data_01_06.shape[0]):
    temp = []
    for i in ['2019년06월_여_10~19세', '2019년06월_여_20~29세', '2019년06월_여_30~39세',
              '2019년06월_여_40~49세', '2019년06월_여_50~59세', '2019년06월_여_60~69세', '2019년06월_여_70~79세']:
        temp.append(change_(num_data_01_06[i].iloc[j]))
    result_age_f_06.append(temp)
    
data_age_f_06 = pd.DataFrame(result_age_f_06, columns = ['f_06_10s','f_06_20s', 'f_06_30s', 'f_06_40s', 'f_06_50s', 'f_06_60s', 'f_06_70s'])

#19.07
result_age_f_07 = []
for j in range(0, num_data_07_12.shape[0]):
    temp = []
    for i in ['2019년07월_여_10~19세', '2019년07월_여_20~29세', '2019년07월_여_30~39세',
              '2019년07월_여_40~49세', '2019년07월_여_50~59세', '2019년07월_여_60~69세', '2019년07월_여_70~79세']:
        temp.append(change_(num_data_07_12[i].iloc[j]))
    result_age_f_07.append(temp)
    
data_age_f_07 = pd.DataFrame(result_age_f_07, columns = ['f_07_10s','f_07_20s', 'f_07_30s', 'f_07_40s', 'f_07_50s', 'f_07_60s', 'f_07_70s'])

#19.08
result_age_f_08 = []
for j in range(0, num_data_07_12.shape[0]):
    temp = []
    for i in ['2019년08월_여_10~19세', '2019년08월_여_20~29세', '2019년08월_여_30~39세',
              '2019년08월_여_40~49세', '2019년08월_여_50~59세', '2019년08월_여_60~69세', '2019년08월_여_70~79세']:
        temp.append(change_(num_data_07_12[i].iloc[j]))
    result_age_f_08.append(temp)

data_age_f_08 = pd.DataFrame(result_age_f_08, columns = ['f_08_10s','f_08_20s', 'f_08_30s', 'f_08_40s', 'f_08_50s', 'f_08_60s', 'f_08_70s'])

#19.09
result_age_f_09 = []
for j in range(0, num_data_07_12.shape[0]):
    temp = []
    for i in ['2019년09월_여_10~19세', '2019년09월_여_20~29세', '2019년09월_여_30~39세',
              '2019년09월_여_40~49세', '2019년09월_여_50~59세', '2019년09월_여_60~69세', '2019년09월_여_70~79세']:
        temp.append(change_(num_data_07_12[i].iloc[j]))
    result_age_f_09.append(temp)
    
data_age_f_09 = pd.DataFrame(result_age_f_09, columns = ['f_09_10s','f_09_20s', 'f_09_30s', 'f_09_40s', 'f_09_50s', 'f_09_60s', 'f_09_70s'])

#19.10
result_age_f_10 = []
for j in range(0, num_data_07_12.shape[0]):
    temp = []
    for i in ['2019년10월_여_10~19세', '2019년10월_여_20~29세', '2019년10월_여_30~39세',
              '2019년10월_여_40~49세', '2019년10월_여_50~59세', '2019년10월_여_60~69세', '2019년10월_여_70~79세']:
        temp.append(change_(num_data_07_12[i].iloc[j]))
    result_age_f_10.append(temp)
    
data_age_f_10 = pd.DataFrame(result_age_f_10, columns = ['f_10_10s','f_10_20s', 'f_10_30s', 'f_10_40s', 'f_10_50s', 'f_10_60s', 'f_10_70s'])

#19.11
result_age_f_11 = []
for j in range(0, num_data_07_12.shape[0]):
    temp = []
    for i in ['2019년11월_여_10~19세', '2019년11월_여_20~29세', '2019년11월_여_30~39세',
              '2019년11월_여_40~49세', '2019년11월_여_50~59세', '2019년11월_여_60~69세', '2019년11월_여_70~79세']:
        temp.append(change_(num_data_07_12[i].iloc[j]))
    result_age_f_11.append(temp)
    
data_age_f_11 = pd.DataFrame(result_age_f_11, columns = ['f_11_10s','f_11_20s', 'f_11_30s', 'f_11_40s', 'f_11_50s', 'f_11_60s', 'f_11_70s'])

#19.12
result_age_f_12 = []
for j in range(0, num_data_07_12.shape[0]):
    temp = []
    for i in ['2019년12월_여_10~19세', '2019년12월_여_20~29세', '2019년12월_여_30~39세',
              '2019년12월_여_40~49세', '2019년12월_여_50~59세', '2019년12월_여_60~69세', '2019년12월_여_70~79세']:
        temp.append(change_(num_data_07_12[i].iloc[j]))
    result_age_f_12.append(temp)
    
data_age_f_12 = pd.DataFrame(result_age_f_12, columns = ['f_12_10s','f_12_20s', 'f_12_30s', 'f_12_40s', 'f_12_50s', 'f_12_60s', 'f_12_70s'])

#20.01
result_age_f_20_01 = []
for j in range(0, num_data_20_01_03.shape[0]):
    temp = []
    for i in ['2020년01월_여_10~19세', '2020년01월_여_20~29세', '2020년01월_여_30~39세',
              '2020년01월_여_40~49세', '2020년01월_여_50~59세', '2020년01월_여_60~69세', '2020년01월_여_70~79세']:
        temp.append(change_(num_data_20_01_03[i].iloc[j]))
    result_age_f_20_01.append(temp)
    
data_age_f_20_01 = pd.DataFrame(result_age_f_20_01, columns = ['f_20_01_10s','f_20_01_20s', 'f_20_01_30s', 'f_20_01_40s', 'f_20_01_50s', 'f_20_01_60s', 'f_20_01_70s'])

#20.02
result_age_f_20_02 = []
for j in range(0, num_data_20_01_03.shape[0]):
    temp = []
    for i in ['2020년02월_여_10~19세', '2020년02월_여_20~29세', '2020년02월_여_30~39세',
              '2020년02월_여_40~49세', '2020년02월_여_50~59세', '2020년02월_여_60~69세', '2020년02월_여_70~79세']:
        temp.append(change_(num_data_20_01_03[i].iloc[j]))
    result_age_f_20_02.append(temp)
    
data_age_f_20_02 = pd.DataFrame(result_age_f_20_02, columns = ['f_20_02_10s','f_20_02_20s', 'f_20_02_30s', 'f_20_02_40s', 'f_20_02_50s', 'f_20_02_60s', 'f_20_02_70s'])
    
#20.03
result_age_f_20_03 = []
for j in range(0, num_data_20_01_03.shape[0]):
    temp = []
    for i in ['2020년03월_여_10~19세', '2020년03월_여_20~29세', '2020년03월_여_30~39세',
              '2020년03월_여_40~49세', '2020년03월_여_50~59세', '2020년03월_여_60~69세', '2020년03월_여_70~79세']:
        temp.append(change_(num_data_20_01_03[i].iloc[j]))
    result_age_f_20_03.append(temp)

data_age_f_20_03 = pd.DataFrame(result_age_f_20_03, columns = ['f_20_03_10s','f_20_03_20s', 'f_20_03_30s', 'f_20_03_40s', 'f_20_03_50s', 'f_20_03_60s', 'f_20_03_70s'])
data_age_f = pd.concat([data_age_f_01, data_age_f_02, data_age_f_03, data_age_f_04, data_age_f_05, data_age_f_06,
                       data_age_f_07, data_age_f_08, data_age_f_09, data_age_f_10, data_age_f_11, data_age_f_12,
                       data_age_f_20_01 ,data_age_f_20_02, data_age_f_20_03], axis=1)
data_age_f.index = ['서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종', '경기', '강원', '충북',
                     '충남', '전북', '전남', '경북', '경남', '제주']

### 2020년04월 여성

In [ ]:
#20.04
result_age_f_target = []
for j in range(0, num_target.shape[0]):
    temp = []
    for i in ['2020년04월_여_10~19세', '2020년04월_여_20~29세', '2020년04월_여_30~39세',
            '2020년04월_여_40~49세', '2020년04월_여_50~59세', '2020년04월_여_60~69세',
            '2020년04월_여_70~79세']:
        temp.append(change_(num_target[i].iloc[j]))
    result_age_f_target.append(temp)
data_f_20_04 = pd.DataFrame(result_age_f_target, columns = ['10s', '20s', '30s', '40s', '50s', '60s', '70s'])
data_f_20_04.index = ['서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종', '경기', '강원', '충북',
                     '충남', '전북', '전남', '경북', '경남', '제주']

## 2019년01월~2020년3월 지역,성별,나이 별 데이터

In [ ]:
data_po = pd.concat([data_age_m, data_age_f], axis = 1)
data_po.index = ['서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종', '경기', '강원', '충북',
                     '충남', '전북', '전남', '경북', '경남', '제주']

## 2020년4월 지역,성별,나이 별 데이터

In [ ]:
data_population_target = pd.concat([data_m_20_04, data_f_20_04], axis =1)
data_population_target.index = ['서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종', '경기', '강원', '충북',
                     '충남', '전북', '전남', '경북', '경남', '제주']
data_population_target.columns = ['20_04_10s','20_04_20s', '20_04_30s', '20_04_40s', '20_04_50s', '20_04_60s','20_04_70s',
                                 'f_20_04_10s', 'f_20_04_20s', 'f_20_04_30s', 'f_20_04_40s', 'f_20_04_50s', 'f_20_04_60s',
                                 'f_20_04_70s']

In [ ]:
# 본 데이터인 data와 합치기 위한 기준을 만들기 위해 년월,자역,나이,성별, 그에 따른 인구수를 담고 있는 리스트 생성
result = []
result_va = []
for j in data_po.index:
    temp = []
    temp_va = []
    for i in data_po.columns:
        place = str(j)
        age = str(i)[-3:]
        
        if len(str(i)) == 5:
            sex = str(1)
            if len(str(i)[:-4]) ==1:
                date = '20190{}'.format(str(i)[:-4])
            value = date + ',' + place + ',' + age + ',' + sex
            
        elif len(str(i)) == 6:
            sex = str(1)
            if len(str(i)[:-4])==2:
                date = '2019{}'.format(str(i)[:-4])
            value = date + ',' + place + ',' + age + ',' + sex
            
        elif len(str(i)) == 9:
            sex = str(1)
            date = '2020{}'.format(str(i)[-6:-4])
            value = date + ',' + place + ',' + age + ',' + sex
            
        elif len(str(i)) == 8:
            sex = str(2)
            date = '2019{}'.format(str(i)[-6:-4])
            value = date + ',' + place + ',' + age + ',' + sex
            
        elif len(str(i)) == 11:
            sex = str(2)
            date = '2020{}'.format(str(i)[-6:-4])
            value = date + ',' + place + ',' + age + ',' + sex
        pop = int(data_po[i].loc[j])
        temp.append(value)
        temp_va.append(pop)

    result.append(temp)
    result_va.append(temp_va)

In [ ]:
# list를 이용하여 DataFrame 생성
population = pd.DataFrame(columns = ['for_population'])
for i in range(0, 17):
    a = pd.DataFrame(result[i],columns = ['for_population'])
    population = pd.concat([population, a], axis = 0)

value = pd.DataFrame(columns = ['population'])
for i in range(0, 17):
    a = pd.DataFrame(result_va[i],columns = ['population'])
    value = pd.concat([value, a], axis = 0)

population = pd.concat([population, value], axis = 1)

In [ ]:
#해당과정은 오래 걸려서 아래의 파일로 저장
population.to_csv('population.csv', index = False)

### 동일한 과정을 2020년04월 데이터에도 적용

In [ ]:
# 2020년4월 데이터도 동일
result = []
result_va = []
for j in data_population_target.index:
    temp = []
    temp_va = []
    for i in data_population_target.columns:
        place = str(j)
        age = str(i)[-3:]
        
        if len(str(i)) == 5:
            sex = str(1)
            if len(str(i)[:-4]) ==1:
                date = '20190{}'.format(str(i)[:-4])
            elif len(str(i)[:-4])==2:
                date = '2019{}'.format(str(i)[:-4])
            value = date + ',' + place + ',' + age + ',' + sex
            
        elif len(str(i)) == 6:
            sex = str(1)
            if len(str(i)[:-4])==2:
                date = '2019{}'.format(str(i)[:-4])
            value = date + ',' + place + ',' + age + ',' + sex
        
        elif len(str(i)) == 9:
            sex = str(1)
            date = '2020{}'.format(str(i)[-6:-4])
            value = date + ',' + place + ',' + age + ',' + sex
            
        elif len(str(i)) == 8:
            sex = str(2)
            date = '2019{}'.format(str(i)[-6:-4])
            value = date + ',' + place + ',' + age + ',' + sex
            
        elif len(str(i)) == 11:
            sex = str(2)
            date = '2020{}'.format(str(i)[-6:-4])
            value = date + ',' + place + ',' + age + ',' + sex
        pop = int(data_population_target[i].loc[j])
        temp.append(value)
        temp_va.append(pop)

    result.append(temp)
    result_va.append(temp_va)

In [ ]:
# 위와 과정 동일
population_target = pd.DataFrame(columns = ['for_population'])
for i in range(0, 17):
    a = pd.DataFrame(result[i],columns = ['for_population'])
    population_target = pd.concat([population_target, a], axis = 0)

value_target = pd.DataFrame(columns = ['population'])
for i in range(0, 17):
    a = pd.DataFrame(result_va[i],columns = ['population'])
    value_target = pd.concat([value_target, a], axis = 0)
    
population_target = pd.concat([population_target, value_target], axis = 1)

In [ ]:
#날짜, 지역, 나이, 성별 - population 데이터와 본 데이터인 data를 병합하기 위해
data['REG_YYMM'] = data['REG_YYMM'].astype(str)
data['CARD_SIDO_NM'] = data['CARD_SIDO_NM'].astype(str)
data['AGE'] = data['AGE'].astype(str)
data['SEX_CTGO_CD'] = data['SEX_CTGO_CD'].astype(str)
data['for_population'] = data['REG_YYMM'] + ',' + data['CARD_SIDO_NM'] + ',' + data['AGE'] + ',' + data['SEX_CTGO_CD']

In [ ]:
data = pd.merge(data, population, how = 'left', on = 'for_population')

In [ ]:
#해당과정은 오래 걸려서 아래의 파일로 저장
population_target.to_csv('population_target', index = False)

# 5. 월/지역에 따른 축제 수
## columns
### - for_fest : '년월,지역'
### - fest_num : 년월,지역에 따른 축제 수
## DataFrame
### - fest : 2019년01월~2020년04월까지의 지역별 축제 수
## Dictionary
### - result : {'년월, 지역' : 조건에 맞는 축제 수}

In [4]:
def crawl_festival_num(x) :
    
    
    festival_data_10 = []
    chrome_driver = '/Users/parkjeongwon/Downloads/chromedriver'
    browser = webdriver.Chrome(chrome_driver)
    url = 'https://www.gov.kr/portal/vfnews?srchSttusCls=ALL&srchAddrCd=ALL&srchStdt=&srchEddt=&srchTxt='
    browser.get(url)
    city = []
    for i in range(2,19):
        elem0=browser.find_element_by_xpath('//*[@id="srchAddrCd"]/option[{}]'.format(i))
        city.append(elem0.text)
        elem0.click()
        temp_num = []
        
        if x == 2019:
            for j in range(1,13):
                
                elem1=browser.find_element_by_id('srchStdt')
                elem1.clear()
                if j <10:
                    elem1.send_keys("20190{}01".format(j))
                else:
                    elem1.send_keys("2019{}01".format(j))



                elem2=browser.find_element_by_id('srchEddt')
                elem2.clear()
                if j <10:
                    elem2.send_keys("20190{}31".format(j))
                else:
                    elem2.send_keys("2019{}31".format(j))
                    
                btn=browser.find_elements_by_xpath('//*[@id="vfnewsVo"]/div[1]/div[2]/p/span/button')
                btn[0].click()            

                src = browser.page_source
                soup = BeautifulSoup(src)


                time.sleep(1)
                fest = soup.select_one('span.sum')
                num_fest = fest.get_text()

                temp_num.append(num_fest)
        
        elif x == 2020:
            for j in range(1,5):
                elem1=browser.find_element_by_id('srchStdt')
                elem1.clear()
                if j <10:
                    elem1.send_keys("20200{}01".format(j))
                else:
                    elem1.send_keys("2020{}01".format(j))



                elem2=browser.find_element_by_id('srchEddt')
                elem2.clear()
                if j <10:
                    elem2.send_keys("20200{}31".format(j))
                else:
                    elem2.send_keys("2020{}31".format(j))
            
            
        

                btn=browser.find_elements_by_xpath('//*[@id="vfnewsVo"]/div[1]/div[2]/p/span/button')
                btn[0].click()            

                src = browser.page_source
                soup = BeautifulSoup(src)


                time.sleep(1)
                fest = soup.select_one('span.sum')
                num_fest = fest.get_text()

                temp_num.append(num_fest)
                
        festival_data_10.append(temp_num)
        
        
    print('success !')
    browser.close()
    
    return festival_data_10, city

In [5]:
#2019년 데이터(1월~ 12월)
fest_num, city = crawl_festival_num(2019) 

success !


In [6]:
#2019년 데이터(1월~ 12월))
# 본 데이터인 data의 값들에 맞게 column, index 변환
fest_num_df = pd.DataFrame(fest_num).transpose()
fest_num_df.columns = ['서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종', '경기', '강원', '충북', '충남', '전북',
                      '전남', '경북', '경남', '제주']
fest_num_df.index = range(201901, 201913)
fest = fest_num_df

In [7]:
#2020년 데이터(1월~4월)
fest_num, city = crawl_festival_num(2020) 

success !


In [8]:
#2020년 데이터(1월~4월)
# 본 데이터인 data의 값들에 맞게 column, index 변환
fest_num_df = pd.DataFrame(fest_num).transpose()
fest_num_df.columns = ['서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종', '경기', '강원', '충북', '충남', '전북',
                      '전남', '경북', '경남', '제주']
fest_num_df.index = range(202001, 202005)

In [9]:
#2019년 2020년 병합(2019년1월부터 2020년4월까지)
fest = pd.concat([fest, fest_num_df], axis = 0)

In [10]:
# 본 데이터인 data에 적용시키는 방법으로 apply를 쓰기 위해 년월,지역을 키값으로, 축제 수를 value로 하는 딕셔너리 생성
result = {}
for i in fest.index:
    for j in fest.columns:
        va = fest[j].loc[i]
        idx = str(j) + ',' + str(i)
        result[idx] = va
print(result)  

{'서울,201901': '1', '부산,201901': '1', '대구,201901': '1', '인천,201901': '1', '광주,201901': '1', '대전,201901': '1', '울산,201901': '1', '세종,201901': '1', '경기,201901': '1', '강원,201901': '1', '충북,201901': '1', '충남,201901': '1', '전북,201901': '1', '전남,201901': '1', '경북,201901': '1', '경남,201901': '1', '제주,201901': '1', '서울,201902': '1', '부산,201902': '1', '대구,201902': '1', '인천,201902': '1', '광주,201902': '1', '대전,201902': '1', '울산,201902': '1', '세종,201902': '1', '경기,201902': '1', '강원,201902': '1', '충북,201902': '1', '충남,201902': '1', '전북,201902': '1', '전남,201902': '1', '경북,201902': '1', '경남,201902': '1', '제주,201902': '1', '서울,201903': '1', '부산,201903': '1', '대구,201903': '1', '인천,201903': '1', '광주,201903': '1', '대전,201903': '1', '울산,201903': '1', '세종,201903': '1', '경기,201903': '1', '강원,201903': '1', '충북,201903': '1', '충남,201903': '1', '전북,201903': '1', '전남,201903': '1', '경북,201903': '1', '경남,201903': '1', '제주,201903': '1', '서울,201904': '1', '부산,201904': '1', '대구,201904': '1', '인천,201904': '1', '광주,20190

In [ ]:
# result의 값들을 적용시키기 위해 새로운 column 생성
data['REG_YYMM'] = data['REG_YYMM'].astype(str)
data['CARD_SIDO_NM'] = data['CARD_SIDO_NM'].astype(str)
data['for_fest'] = data['CARD_SIDO_NM'] + ',' +  data['REG_YYMM'] 

In [ ]:
data['fest_num'] = data['for_fest'].apply(lambda x : result[x])

# 6. 월별 휴일 수(주말 + 대체공휴일 + 쉬는날)
## columns
### - rest_num : 월별 축제 수
## Dictionary
### - rest : 2019년01월~2020년04월,2020년07월의 휴일 수

In [11]:
# 달력 보고 계산
rest = {'201901' : 9,
       '201902' : 11,
       '201903' : 11,
       '201904' : 8,
       '201905' : 9,
       '201906' : 11,
       '201907' : 8,
       '201908' : 10,
       '201909' : 11,
       '201910' : 10,
       '201911' : 9,
       '201912' : 10,
       '202001' : 11,
       '202002' : 9,
       '202003' : 9,
       '202004' : 10,
       '202007' : 8}

In [ ]:
data['rest_num'] = data['REG_YYMM'].apply(lambda x : rest[x])

In [12]:
data = pd.read_csv('perfect.csv')

# 9. 2020년04월에 합칠 데이터
## DataFrame
### data_for_20_04_last_month : data[['last_month', 'last_month_AMT_r']] -> 전달 카드사용량
### data_for_20_04 : 전반적인 데이터
### population_target : for_population(년월, 지역, 나이, 성별)을 기준으로 인구수를 병합
## Columns, Dictionary
### fest_num : for_fest(지역,년월)을 기준으로 함, result 딕셔너리에 들어있다.
### rest_num : REG_YYMM(년월)을 기준으로 함, rest 딕셔너리에 들어있다.
## 2020년04월의 데이터가 주어지면 [년월, 지역, 업종]을 제외한 나머지를 제거하고 month 함수를 이용해 month column을 만들어내고 for_20_04[월,지역,업종]을 합친 새로운 column을 만들어내고 data_for_20_04와 for_20_04를 기준으로 병합 후 population,fest_num 등 추가

In [13]:
# 전달 카드사용량 병합을 위한 자료
data_for_20_04_last_month = data[['last_month', 'last_month_AMT_r']]

In [14]:
# 중복 행 제거
data_for_20_04_last_month = data_for_20_04_last_month.drop_duplicates()

# 2020년04월 데이터도 위의 방법으로 가공

In [15]:
test_20_04 = pd.read_csv('/Users/parkjeongwon/Downloads/202004.csv')

In [16]:
# 전달 카드사용량 계산을 위해
test_20_04['REG_YYMM'] = test_20_04['REG_YYMM'].astype(str)
test_20_04['STD_CLSS_NM'] = test_20_04['STD_CLSS_NM'].astype(str)
test_20_04['CARD_SIDO_NM'] = test_20_04['CARD_SIDO_NM'].astype(str)
test_20_04['last_month'] = test_20_04['REG_YYMM'] + ',' + test_20_04['STD_CLSS_NM'] + ',' + test_20_04['CARD_SIDO_NM']

In [17]:
# 결측치를 업종,지역 별 평균으로 대체하기 위해
data_for_isna = data[['CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
test_20_04_for_isna = test_20_04[['CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]

In [18]:
# 2019.01~2020.03데이터 + 2020.04데이터
for_isna = pd.concat([data_for_isna,test_20_04_for_isna], axis = 0)

In [19]:
# 지역, 업종별 평균 구하기
for_isna['CARD_SIDO_NM'] = for_isna['CARD_SIDO_NM'].astype(str)
for_isna['STD_CLSS_NM'] = for_isna['STD_CLSS_NM'].astype(str)

for_isna['for_last'] = for_isna['STD_CLSS_NM'] + ',' + for_isna['CARD_SIDO_NM']

a = for_isna.groupby(['for_last'])['AMT'].agg('mean').reset_index()
a.columns = ['for_last', 'last_month_AMT_r']


for_isna = pd.merge(for_isna, a, how = 'left', on = 'for_last')

In [20]:
for_isna = for_isna[['for_last','last_month_AMT_r']]
for_isna = for_isna.drop_duplicates()

In [21]:
# 19.01~20.03의 전달 카드사용량의 결측치 처리를 이미 위에서 진행했기 때문에 202004의 데이터의 결측치 처리만 하면 된다.
for_isna['for_last'] = for_isna['for_last'].astype(str)

for_isna['last_month'] = '202004' + ',' + for_isna['for_last']

for_isna = for_isna.drop(['for_last'],axis=1)

In [22]:
data_for_20_04_last_month = pd.concat([data_for_20_04_last_month, for_isna], axis = 0)

In [23]:
test_20_04 = pd.merge(test_20_04, data_for_20_04_last_month, how = 'left', on = 'last_month')

In [24]:
#결측치 확인(위와 같이 지역,업종의 평균으로 대체)
test_20_04.isna().sum()

REG_YYMM                0
CARD_SIDO_NM            0
CARD_CCG_NM         10316
STD_CLSS_NM             0
HOM_SIDO_NM             0
HOM_CCG_NM          14894
AGE                     0
SEX_CTGO_CD             0
FLC                     0
CSTMR_CNT               0
AMT                     0
CNT                     0
last_month              0
last_month_AMT_r        0
dtype: int64

In [25]:
# Month Column 생성
test_20_04['month'] = test_20_04['REG_YYMM'].apply(lambda x : month(x))

In [26]:
# 월 나이 성별 / 업종에 쓴 돈의 비율, 지역에 쓴 돈의 비율의 분모를 구하기 위해
test_20_04['month'] = test_20_04['month'].astype(str)
test_20_04['AGE'] = test_20_04['AGE'].astype(str)
test_20_04['SEX_CTGO_CD'] = test_20_04['SEX_CTGO_CD'].astype(str)

test_20_04['standard'] = test_20_04['month'] + ',' + test_20_04['AGE'] + ',' + test_20_04['SEX_CTGO_CD']

# 월 나이 성별 업종 / 업종에 쓴 돈의 비율의 분자를 구하기 위해
test_20_04['month'] = test_20_04['month'].astype(str)
test_20_04['AGE'] = test_20_04['AGE'].astype(str)
test_20_04['SEX_CTGO_CD'] = test_20_04['SEX_CTGO_CD'].astype(str)
test_20_04['STD_CLSS_NM'] = test_20_04['STD_CLSS_NM'].astype(str)

test_20_04['standard_STD_CLSS_NM'] = test_20_04['month'] + ',' + test_20_04['AGE'] + ',' + test_20_04['SEX_CTGO_CD'] + ',' + test_20_04['STD_CLSS_NM']

# 월 나이 성별 지역 / 지역에 쓴 돈의 비율의 분자를 구하기 위해
test_20_04['month'] = test_20_04['month'].astype(str)
test_20_04['AGE'] = test_20_04['AGE'].astype(str)
test_20_04['SEX_CTGO_CD'] = test_20_04['SEX_CTGO_CD'].astype(str)
test_20_04['CARD_SIDO_NM'] = test_20_04['CARD_SIDO_NM'].astype(str)

test_20_04['standard_CARD_SIDO_NM'] = test_20_04['month'] + ',' + test_20_04['AGE'] + ',' + test_20_04['SEX_CTGO_CD'] + ',' + test_20_04['CARD_SIDO_NM']


In [27]:
# 월/나이/성별에 따른 AMT
a = test_20_04.groupby(['standard'])['AMT'].agg('sum').reset_index()
a.columns = ['standard', 'standard_AMT']

test_20_04 = pd.merge(test_20_04, a, how='left', on='standard')

# 월/나이/성별에 따른 업종별 AMT
a = test_20_04.groupby(['standard_STD_CLSS_NM'])['AMT'].agg('sum').reset_index()
a.columns = ['standard_STD_CLSS_NM', 'standard_STD_CLSS_NM_AMT']

test_20_04 = pd.merge(test_20_04, a, how='left', on='standard_STD_CLSS_NM')

# 월/나이/성별에 따른 지역별 AMT
a = test_20_04.groupby(['standard_CARD_SIDO_NM'])['AMT'].agg('sum').reset_index()
a.columns = ['standard_CARD_SIDO_NM', 'standard_CARD_SIDO_AMT']

test_20_04 = pd.merge(test_20_04, a, how='left', on='standard_CARD_SIDO_NM')

# 계산을 위해 정수로 바꿔주기
test_20_04['standard_AMT'] = test_20_04['standard_AMT'].astype(int)
test_20_04['standard_STD_CLSS_NM_AMT'] = test_20_04['standard_STD_CLSS_NM_AMT'].astype(int)
test_20_04['standard_CARD_SIDO_AMT'] = test_20_04['standard_CARD_SIDO_AMT'].astype(int)

#월 / 나이 / 성별에 따른 업종별 비율
test_20_04['standard_STD_CLSS_ratio'] = test_20_04['standard_STD_CLSS_NM_AMT'] / test_20_04['standard_AMT']

#월 / 나이 / 성별에 따른 지역별 비율
test_20_04['standard_CARD_SIDO_ratio'] = test_20_04['standard_CARD_SIDO_AMT'] / test_20_04['standard_AMT']

In [28]:
#2020년 4월 지역, 나이, 성별에 따른 인구 수 가져오기
pop_t = pd.read_csv('population_target')

In [29]:
#test_20_04와 merge 할 때 필요한 기준 만들기
test_20_04['REG_YYMM'] = test_20_04['REG_YYMM'].astype(str)
test_20_04['CARD_SIDO_NM'] = test_20_04['CARD_SIDO_NM'].astype(str)
test_20_04['AGE'] = test_20_04['AGE'].astype(str)
test_20_04['SEX_CTGO_CD'] = test_20_04['SEX_CTGO_CD'].astype(str)

test_20_04['for_population'] = test_20_04['REG_YYMM'] + ',' + test_20_04['CARD_SIDO_NM'] + ',' + test_20_04['AGE'] + ',' + test_20_04['SEX_CTGO_CD']


In [30]:
test_20_04 = pd.merge(test_20_04, pop_t, how = 'left', on = 'for_population')

In [31]:
# test_20_04에 지역별 축제수를 merge 하기 위한 기준
test_20_04['REG_YYMM'] = test_20_04['REG_YYMM'].astype(str)
test_20_04['CARD_SIDO_NM'] = test_20_04['CARD_SIDO_NM'].astype(str)
test_20_04['for_fest'] = test_20_04['CARD_SIDO_NM'] + ',' +  test_20_04['REG_YYMM'] 

In [32]:
test_20_04['fest_num'] = test_20_04['for_fest'].apply(lambda x : result[x])

In [33]:
test_20_04['rest_num'] = test_20_04['REG_YYMM'].apply(lambda x : rest[x])

# 10. 현재 우리가 구하고자하는 월, 지역, 업종, 나이, 성별에 따른 AMT(Target을 구하기 위한 예비 target)
## Columns
### AMT_r : 월, 지역, 업종, 나이, 성별에 따른 AMT

In [34]:
# train_data
data['month'] = data['month'].astype(str)
data['CARD_SIDO_NM'] = data['CARD_SIDO_NM'].astype(str)
data['STD_CLSS_NM'] = data['STD_CLSS_NM'].astype(str)
data['AGE'] = data['AGE'].astype(str)
data['SEX_CTGO_CD'] = data['SEX_CTGO_CD'].astype(str)

data['for_AMT'] = data['month'] + ',' +  data['CARD_SIDO_NM'] + ',' +  data['STD_CLSS_NM'] + ',' +  data['AGE'] + ',' + data['SEX_CTGO_CD']

b = data[['for_AMT', 'AMT']]
b = b.groupby(['for_AMT'])['AMT'].agg('sum').reset_index()
b.columns = ['for_AMT', 'AMT_r']

data = pd.merge(data, b ,how ='left', on ='for_AMT')

In [35]:
#test_data
test_20_04['month'] = test_20_04['month'].astype(str)
test_20_04['CARD_SIDO_NM'] = test_20_04['CARD_SIDO_NM'].astype(str)
test_20_04['STD_CLSS_NM'] = test_20_04['STD_CLSS_NM'].astype(str)
test_20_04['AGE'] = test_20_04['AGE'].astype(str)
test_20_04['SEX_CTGO_CD'] = test_20_04['SEX_CTGO_CD'].astype(str)

test_20_04['for_AMT'] = test_20_04['month'] + ',' +  test_20_04['CARD_SIDO_NM'] + ',' +  test_20_04['STD_CLSS_NM'] + ',' +  test_20_04['AGE'] + ',' + test_20_04['SEX_CTGO_CD']

b = test_20_04[['for_AMT', 'AMT']]
b = b.groupby(['for_AMT'])['AMT'].agg('sum').reset_index()
b.columns = ['for_AMT', 'AMT_r']

test_20_04 = pd.merge(test_20_04, b ,how ='left', on ='for_AMT')

# 11. 예측에 필요한 columns

In [36]:
#train_data
train_data = data[['standard_STD_CLSS_ratio','standard_CARD_SIDO_ratio','last_month_AMT_r',
                           'population','rest_num','fest_num', 'AMT_r']]

In [37]:
#test_data
test_data = test_20_04[['standard_STD_CLSS_ratio','standard_CARD_SIDO_ratio','last_month_AMT_r',
                           'population','rest_num','fest_num', 'AMT_r']]

In [38]:
train_data.to_csv('train_data')
test_data.to_csv('test_data')

# 12. 2019년01월~2020년03월 자료 -> train, 2020년04월 자료 -> test

In [39]:
# train, test 동일
feature = train_data.columns.difference(['AMT_r'])

In [50]:
train_data = pd.read_csv('train_data_1')
test_data = pd.read_csv('test_data_1')

In [51]:
train_x = train_data[feature]
train_y = np.log1p(train_data['AMT_r'])
test_x = test_data[feature]
test_y = np.log1p(test_data['AMT_r'])

In [53]:
test_x.fest_num = test_x.fest_num.astype(int)
train_x[['standard_STD_CLSS_ratio','standard_CARD_SIDO_ratio']] = train_x[['standard_STD_CLSS_ratio','standard_CARD_SIDO_ratio']].astype(float)
test_x[['standard_STD_CLSS_ratio','standard_CARD_SIDO_ratio']] = test_x[['standard_STD_CLSS_ratio','standard_CARD_SIDO_ratio']].astype(float)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [42]:
import lightgbm as lgb

/opt/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [54]:
lgb_dtrain = lgb.Dataset(data = train_x, label = train_y)
lgb_param = {'max_depth': 20, 
            'learning_rate': 0.0001, 
            'n_estimators': 100, 
            'objective': 'regression'}
lgb_model = lgb.train(params = lgb_param, train_set = lgb_dtrain)

/opt/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [55]:
pred_bf = lgb_model.predict(test_x)

In [56]:
# 여기까지가 target을 월,지역,업종,나이,성별에 따른 카드사용량으로 했을 경우
test = np.exp(test_y) - 1
pred = np.exp(pred_bf) - 1
score = np.sqrt(mean_squared_log_error(test, pred))
print(score)

1.8654506642487934


In [57]:
test_20_04['pred'] = pred

In [58]:
a = test_20_04.groupby(['last_month'])['pred'].agg('sum').reset_index()
a.columns = ['last_month', 'pred']

b = test_20_04.groupby(['last_month'])['AMT'].agg('sum').reset_index()
b.columns = ['last_month', 'real']

In [59]:
score = np.sqrt(mean_squared_log_error(a.pred, b.real))

In [60]:
score

7.34407267575874